In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [11]:
folders = os.listdir('../data/raw/final_project/calib')

max_nb = 15

for folder in folders:
    images = os.listdir('../data/raw/final_project/calib/'+folder+'/data')
    
    objpoints = []
    imgpoints = []
    
    im = cv2.imread('../data/raw/final_project/calib/'+folder+'/data/'+images[0])
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    for v,h in zip([5,7,5], [7,11,15]): #np.flip(np.mgrid[5:max_nb+1,5:max_nb+1].T.reshape(-1,2)):
        ret = True
        while ret:
            ret, corners = cv2.findChessboardCorners(im_gray, (v,h))
            #if not ret: ret, corners = cv2.findChessboardCornersSB(im_gray, (v,h))
            if not ret: break
            
            imgpoints.append(corners)
            objp = np.zeros((h*v,3), np.float32)
            objp[:,:2] = np.mgrid[0:v,0:h].T.reshape(-1,2)
            objpoints.append(objp)
            
            start_x = int(np.min(corners[:,0].T[0]))
            end_x = int(np.max(corners[:,0].T[0]))
            start_y = int(np.min(corners[:,0].T[1]))
            end_y = int(np.max(corners[:,0].T[1]))
            im_gray = cv2.rectangle(im_gray, (start_x, start_y), (end_x, end_y), 0, -1)

    cv2.imshow('im', im_gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, im_gray.shape[::-1], None, None)

    if not os.path.exists('../data/undistorted/'):
        os.makedirs('../data/undistorted/')
    for image in images:
        im = cv2.imread('../data/raw/final_project/calib/'+folder+'/data/'+image)
        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        h,w = im.shape[:2]
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
        dst = cv2.undistort(im, mtx, dist, None, newcameramtx)

        if not os.path.exists('../data/undistorted/'+os.path.split(folder)[-1]):
            os.makedirs('../data/undistorted/'+os.path.split(folder)[-1])
        cv2.imwrite('../data/undistorted/'+os.path.split(folder)[-1]+'/'+os.path.split(image)[-1], dst)